<a href="https://colab.research.google.com/github/AndyH96/Atmospheric-emissions-prediction/blob/main/andy_ho_bi_engineering_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing sqlite3 to access the necessary functions for SQL



In [122]:
import sqlite3
conn = sqlite3.connect('assessment.db')
cursor = conn.cursor()
# Drop the table if it exists
cursor.execute("DROP TABLE IF EXISTS Trades")
cursor.execute("DROP TABLE IF EXISTS Traders")
cursor.execute("DROP TABLE IF EXISTS Deals")
cursor.execute("DROP TABLE IF EXISTS Profit")

# Creating necessary datasets

In [123]:
conn.execute ('''
CREATE TABLE Trades (
      Tradenumber varchar(4) NOT NULL,
      MatchId varchar(2) NOT NULL,
      Volume int NOT NULL,
      Price decimal(4, 2) NOT NULL,
      Direction varchar(4) NOT NULL,
PRIMARY KEY (Tradenumber)
); '''
)
conn.commit()
print('Trades table created')

Trades table created


In [124]:
conn.execute ('''
CREATE TABLE Traders (
      TraderCode varchar(2) NOT NULL,
      TraderName varchar(20) NOT NULL,
      TraderTarget int NOT NULL,
PRIMARY KEY (TraderCode)
); '''
)
conn.commit()
print('Traders table created')

Traders table created


In [125]:
conn.execute ('''
CREATE TABLE Deals (
      MatchId varchar(2) NOT NULL,
      MatchedDate date NOT NULL,
      TraderCode varchar(2) NOT NULL,
PRIMARY KEY (MatchId),
FOREIGN KEY (TraderCode) REFERENCES Traders(TraderCode)
); '''
)
conn.commit()
print('Deals table created')

Deals table created


In [126]:
conn.execute("INSERT INTO Trades (Tradenumber, MatchId, Volume, Price, Direction) VALUES \
  ('T001', 'M1', 1000, 0.10, 'Buy'),\
  ('T002', 'M2', 100, 0.12, 'Buy'),\
  ('T003', 'M3', 400, 0.11, 'Buy'),\
  ('T004', 'M4', 1500, 0.09, 'Buy'),\
  ('T005', 'M5', 600, 0.13, 'Buy'),\
  ('T006', 'M3', 400, 0.15, 'Sell'),\
  ('T007', 'M2', 100, 0.16, 'Sell'),\
  ('T008', 'M5', 600, 0.14, 'Sell'),\
  ('T009', 'M4', 1500, 0.12, 'Sell'),\
  ('T010', 'M1', 1000, 0.12, 'Sell')")
conn.commit()

print('Trades table created')

Trades table created


In [127]:
conn.execute("INSERT INTO Traders (TraderCode, TraderName, TraderTarget) VALUES \
  ('AB', 'Arber', 40), \
  ('RA', 'Riccardo', 20), \
  ('CM', 'Cornelius', 20)")

conn.commit()

print('Traders table created')

Traders table created


In [128]:
conn.execute("INSERT INTO Deals (MatchId, MatchedDate, TraderCode) VALUES \
  ('M1', '2023-04-01', 'RA'),\
  ('M2', '2023-04-01', 'AB'),\
  ('M3', '2023-04-03', 'CM'),\
  ('M4', '2023-04-04', 'AB'),\
  ('M5', '2023-04-06', 'RA')")

conn.commit()

print('Deals table created')

Deals table created


# Task 1: SQL Skills


Question 1: Calculate the total sold volume.

In [129]:
conn = sqlite3.connect('assessment.db')
cursor = conn.execute('''
SELECT SUM(volume) as TotalSoldVolume 
FROM Trades 
WHERE Direction = 'Sell'
''')
for row in cursor:
  print('The total volume sold is:', row[0])

The total volume sold is: 3600


Question 2: Calculate the weighted average buy price.

In [130]:
conn = sqlite3.connect('assessment.db')
cursor = conn.execute('''
SELECT SUM(volume * price) / SUM(volume) as WeightedAverageBuyPrice 
FROM Trades 
WHERE Direction = 'Buy'

''')
for row in cursor:
  print('The weighted average buy price is:', row[0])


The weighted average buy price is: 0.1025


Question 3: Calculate the profit per deal.

In [131]:
# Creating a profit per deal table as it could be multi-functional 
conn.execute ('''
CREATE TABLE Profit (
  MatchId VARCHAR(10),
  ProfitPerDeal DECIMAL(10,2)

); '''
)
conn.commit()
print('Profit table created')


Profit table created


In [132]:
conn.execute('''
    INSERT INTO Profit (MatchId, ProfitPerDeal)
    SELECT DISTINCT MatchId, (SUM(CASE WHEN Direction = 'Buy' THEN -1 ELSE 1 END * volume * price)) as ProfitPerDeal
    FROM Trades
    GROUP BY MatchId
''')

cursor = conn.execute('SELECT * FROM Profit')
for row in cursor:
  print('MatchId:', row[0], 'Profit Per Deal:', row[1])

MatchId: M1 Profit Per Deal: 20
MatchId: M2 Profit Per Deal: 4
MatchId: M3 Profit Per Deal: 16
MatchId: M4 Profit Per Deal: 45
MatchId: M5 Profit Per Deal: 6.000000000000014


Question 5: Identify the trader who did the second most profit.

In [133]:
cursor = conn.execute('''
WITH second_most_profit AS (
    SELECT  
        t.TraderCode, 
        t.TraderName,
        SUM(p.ProfitPerDeal) AS TotalProfit,
        RANK() OVER (ORDER BY SUM(p.ProfitPerDeal) DESC) AS row
    FROM Profit AS p
    LEFT JOIN Deals AS d ON d.MatchId = p.MatchId
    LEFT JOIN Traders AS t ON t.TraderCode = d.TraderCode
    GROUP BY t.TraderCode, t.TraderName
)
SELECT TraderName, TraderCode, TotalProfit
FROM second_most_profit
WHERE row = 2

''')
for row in cursor:
    print('The trader with the second most profit is:', row[0], 'TraderName:', row[1], 'TotalProfit', row[2])


The trader with the second most profit is: Riccardo TraderName: RA TotalProfit 26.000000000000014


Question 6: Calculate the total profit per every day of the period between the 1st and the 10th of April, both included.

In [134]:
cursor = conn.execute('''
WITH daily_profit AS ( 
    SELECT date(MatchedDate) AS Date, 
       SUM(p.ProfitPerDeal) AS TotalProfit
FROM Profit AS p
    LEFT JOIN Deals AS d ON d.MatchId = p.MatchId
    LEFT JOIN Traders AS t ON t.TraderCode = d.TraderCode
WHERE MatchedDate BETWEEN '2023-04-01' AND '2023-04-10'
GROUP BY date(MatchedDate)
)
SELECT Date,
       SUM(TotalProfit) AS DailyProfit
FROM daily_profit
GROUP BY Date
''')

for row in cursor:
    print('Date:', row[0], 'DailyProfit:', row[1])


Date: 2023-04-01 DailyProfit: 24
Date: 2023-04-03 DailyProfit: 16
Date: 2023-04-04 DailyProfit: 45
Date: 2023-04-06 DailyProfit: 6.000000000000014


Question 7: Calculate the cumulative profit for the period between the first and the 10th of April, both included.

In [135]:
cursor = conn.execute('''
WITH daily_profit AS ( 
    SELECT date(MatchedDate) AS Date, 
    SUM(p.ProfitPerDeal) AS DailyProfit
    FROM Profit AS p
    LEFT JOIN Deals AS d ON d.MatchId = p.MatchId
    LEFT JOIN Traders AS t ON t.TraderCode = d.TraderCode
    WHERE MatchedDate BETWEEN '2023-04-01' AND '2023-04-10'
    GROUP BY date(MatchedDate)
),
cumulative_profit AS (
    SELECT Date, 
    SUM(DailyProfit) OVER (ORDER BY Date) AS CumulativeProfit
    FROM daily_profit
)
SELECT Date, CumulativeProfit
FROM cumulative_profit
''')

for row in cursor:
    print('Date:', row[0], 'Cumulative Profit To Date:', row[1])

Date: 2023-04-01 Cumulative Profit To Date: 24
Date: 2023-04-03 Cumulative Profit To Date: 40
Date: 2023-04-04 Cumulative Profit To Date: 85
Date: 2023-04-06 Cumulative Profit To Date: 91.00000000000001
